In [1]:
!pip install -q kaggle
!pip install spacy
!python -m spacy download en_core_web_md
!pip install Unidecode
!pip install word2number
!pip install contractions
!pip install nltk
!pip install pandas
!pip install sklearn
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.5 MB 415 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 22.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5582 sha256=fd9397f632917412a9d8040700f0b35aae9fce4b46b5155508af1cd999480c92
  Stored in directory: /root/.cache/pip/wheels/4b/c3/77/a5f48aeb0d3efb7cd5ad61cbd3da30bbf9ffc9662b07c9f879
Successfully built word2number
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/s

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
import os

In [ ]:
! mkdir ~/.kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sushanth00reddy","key":"0904f90198d8881cdbe3024145cdeef2"}'}

In [ ]:
!cp kaggle.json ~/.kaggle/

! kaggle datasets list

In [ ]:
!kaggle competitions download -c nlp-getting-started
!unzip /content/nlp-getting-started.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import numpy as np
import pandas as pd


FILE_NAME1="spambase.data"
#data=np.genfromtxt(FILE_NAME,delimiter=",")
df = pd.read_csv('/content/train.csv')


df=df.drop(['location'], axis=1)

df = df.drop(['id'], axis=1)
df =df.drop(['keyword'], axis=1)
print(df.sample(5))
print(df.info())
print(df.iloc[0:21])


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go


sns.countplot(x='target', data=df)
plt.gca().set_ylabel('tweets')


In [ ]:
print(df['text'])

In [ ]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
import nltk
import string
import re

from nltk.tokenize import word_tokenize
nlp = spacy.load('en_core_web_md')
#print(type(nlp))


def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text


def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())


def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text


def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text
def punct_add_space(x):
    x = str(x)
    for punct in string.punctuation:
        x = x.replace(punct, f' {punct} ')
    return x

def remove_msg_punctuations(text):
	puntuation_removed_msg = "".join([word for word in text if word not in string.punctuation ])
	return puntuation_removed_msg
def remove_special_characters(word):
    result=word.translate(str.maketrans(dict.fromkeys(string.punctuation)))
    return result

def clean_text(text):
  text = text.lower()
  text = strip_html_tags(text)
  text = remove_whitespace(text)
  text = remove_accented_chars(text)
  text = expand_contractions(text)
  text = punct_add_space(text)
  text = remove_msg_punctuations(text)
  text = re.sub("[^a-z' ]+", ' ', text)
  text = re.sub('http\S+', ' ', text)
  text = remove_special_characters(text)
  return text.split()
  

df['text'] = df['text'].apply(lambda x: clean_text(x))

print(df.sample(5))


                                                   text  target
6270  [looks, like, a, perfect, storm, free, evening...       1
3383  [my, school, just, put, the, evacuation, alarm...       0
5905  [watch, this, airport, get, swallowed, up, by,...       1
2296  [jolly, jinu, you, said, they, are, terrorist,...       0
5810  [refuse, to, let, my, life, be, reduced, to, r...       0


# New Section

In [ ]:

new_stop_words = [ 'the', 'a', 'in', 'to', 'of', 'i', 'and', 'is', 'you', 'for', 'on', 'it', 'my', 'that',
               'with', 'are', 'at', 'by', 'this', 'have', 'from', 'be', 'was', 'do', 'will', 'as', 'up', 
               'me', 'am', 'so', 'we', 'your', 'has', 'when', 'an', 's', 'they', 'about', 'been', 'there',
               'who', 'would', 'into', 'his', 'them', 'did', 'w', 'their', 'm', 'its', 'does', 'where', 'th',
               'b', 'd', 'x', 'p', 'o', 'r', 'c', 'n', 'e', 'g', 'v', 'k', 'l', 'f', 'j', 'z', 'us', 'our',
               'all', 'can', 'may' ] 
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
deselect_stop_words = ['no', 'not']
for words in stop_words.copy():
  if words in deselect_stop_words:
    stop_words.remove(words)
for words in stop_words.copy():
  if words in new_stop_words:
    stop_words.add(words)
#print(stop_words)
def remove_stop_words(words):
  result = []
  for word in words:
    if not (word in stop_words):
      result.append(word)
  return result
df['text'] = df['text'].apply(lambda words: remove_stop_words(words))
print(df.iloc[:5])


                                                text  target
0  [deeds, reason, earthquake, may, allah, forgiv...       1
1      [forest, fire, near, la, ronge, sask, canada]       1
2  [residents, asked, shelter, place, notified, o...       1
3  [people, receive, wildfires, evacuation, order...       1
4  [got, sent, photo, ruby, alaska, smoke, wildfi...       1


In [ ]:
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
def lemmatize(words):
  lems =[]
  for w in words:
    lems.append(lemmatizer.lemmatize(w)) 
  return lems
df['text'] = df['text'].apply(lambda words: lemmatize(words))


print(df.iloc[:5])


                                                text  target
0  [deed, reason, earthquake, may, allah, forgive...       1
1      [forest, fire, near, la, ronge, sask, canada]       1
2  [resident, asked, shelter, place, notified, of...       1
3  [people, receive, wildfire, evacuation, order,...       1
4  [got, sent, photo, ruby, alaska, smoke, wildfi...       1


In [ ]:
"""from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def stem(words):
  stems = []
  for w in words:
    stems.append(porter.stem(w))
    return stems
df['text'] = df['text'].apply(lambda words: stem(words))

print(df.iloc[:5])"""

"from nltk.stem.porter import PorterStemmer\nporter = PorterStemmer()\ndef stem(words):\n  stems = []\n  for w in words:\n    stems.append(porter.stem(w))\n    return stems\ndf['text'] = df['text'].apply(lambda words: stem(words))\n\nprint(df.iloc[:5])"

In [ ]:

!wget -q http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip -q glove.twitter.27B.zip

^C
replace glove.twitter.27B.25d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
text_embedding_dimension = 200
path_to_glove_file = '/content/glove.twitter.27B.{}d.txt'.format(text_embedding_dimension)
embeddings_index_200 = {}

with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index_200[word] = coefs

unknown_words = []
total_words = 0

def find_unknown_words(words):
    
    global total_words
    total_words = total_words + len(words)
    
    for word in words:
        if not (word in embeddings_index_200):
            unknown_words.append(word)
    
    return words


df['text'].apply(lambda words: find_unknown_words(words))

print( f'{len(unknown_words)/total_words*100} % of words are unknown' )
print(len(unknown_words))


10.439740703449369 % of words are unknown
9228


In [ ]:
def analyze_unknown_words(unknown_words):
    
    unknown_words = np.array(unknown_words)
    (word, count) = np.unique(unknown_words, return_counts=True)
    
    word_freq = pd.DataFrame({'word': word, 'count': count}).sort_values('count', ascending=False)

    fig = go.Figure(data=[go.Table(
          header=dict(values=list(word_freq.columns),
                    fill_color='paleturquoise',
                    align='left'),
          cells=dict(values=[word_freq['word'], word_freq['count']],
                    fill_color='lavender',
                    align='left'))
          ])
    fig.update_layout(width=300, height=300, margin={'b':0, 'l':0, 'r':0, 't':0, 'pad':0})
    fig.show()
        
analyze_unknown_words(unknown_words)

In [ ]:
# cross_validation_ratio = 0.2
cross_validation_ratio = 0.05

mask = np.random.rand(len(df)) > cross_validation_ratio

train_df = df[mask]

val_df = df[~mask]

In [ ]:
x_train_text = train_df['text'].values
x_val_text = val_df['text'].values
y_train = train_df['target'].values
y_val = val_df['target'].values


In [ ]:
print(df.iloc[19])

text      [goooooooaaaaaal]
target                    0
Name: 19, dtype: object


In [ ]:
def text_embed(words):
    
    unknown_indices = []
    mean = np.zeros(text_embedding_dimension)
    
    for i in range(len(words)):
        if words[i] in embeddings_index_200:
            words[i] = embeddings_index_200[ words[i] ]
            mean += words[i]
        else:
            unknown_indices.append(i)
            
    if(len(words)-len(unknown_indices)!=0):
      mean /= len(words)-len(unknown_indices)
    
    # unknown words in the text are represented using the mean of the known words
    for i in unknown_indices:
        words[i] = mean
    
    return np.array(words)

df['text'] = df['text'].apply(lambda words: text_embed(words))


In [ ]:
print(df.iloc[19])

text      [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
target                                                    0
Name: 19, dtype: object


In [ ]:

print(df.info)

<bound method DataFrame.info of                                                    text  target
0     [[0.26635, 0.76221, -0.39284, 0.033448, -0.854...       1
1     [[-0.4864, 0.07589, 0.11724, 0.017693, -0.2309...       1
2     [[-0.14397, 0.17361, -0.60361, -0.08003, 0.228...       1
3     [[0.011112, 0.4785, -0.092164, 0.010874, -0.06...       1
4     [[0.19383, 0.45681, -0.23766, -0.010523, 0.113...       1
...                                                 ...     ...
7608  [[0.45059001445770264, -0.5434399843215942, -0...       1
7609  [[-0.257889986038208, -0.010452000424265862, 0...       1
7610  [[0.41319000720977783, 0.1357399970293045, -0....       1
7611  [[-0.2656, 0.25355, -1.2521, 0.6124, 0.12742, ...       1
7612  [[0.24222999811172485, -0.0492670014500618, -0...       1

[7613 rows x 2 columns]>


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
torch.manual_seed(103)
torch.cuda.manual_seed(103)
np.random.seed(103)
deviceCount = torch.cuda.device_count()
print(deviceCount)

cuda0 = None
if deviceCount > 0:
  print(torch.cuda.get_device_name(0))
  cuda0 = torch.device('cuda:0')

0


In [ ]:
class LSTMnetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden_size = 50
        self.input_size = text_embedding_dimension
        self.num_layers = 1
        self.bidirectional = True
        self.num_directions = 1
        self.dropout1 = nn.Dropout(p=0.3)

        if self.bidirectional:
            self.num_directions = 2
 
        self.lstm = nn.LSTM( self.input_size, self.hidden_size, self.num_layers, 
                             bidirectional=self.bidirectional )
        self.linear = nn.Linear(self.hidden_size*self.num_directions,1)

    def forward(self, tweet):
        
        lstm_out, _ = self.lstm( tweet.view(len(tweet), 1, -1) )

        x = self.dropout1( lstm_out.view(len(tweet),-1) )
        
        output = self.linear(x)
        
        pred = torch.sigmoid( output[-1] )
        
        return pred


In [ ]:
lstm_model = LSTMnetwork()

if cuda0 != None:
  lstm_model.to(cuda0)

criterion_text = nn.BCELoss()
optimizer_text = torch.optim.Adam(lstm_model.parameters(), lr=0.001)
scheduler_text = torch.optim.lr_scheduler.ExponentialLR(optimizer_text, gamma=0.1)

In [ ]:
lstm_model

LSTMnetwork(
  (dropout1): Dropout(p=0.3, inplace=False)
  (lstm): LSTM(200, 50, bidirectional=True)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [ ]:

lstm_model_weight = 1

In [ ]:

start_time = time.time()

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(4):  

    epoch_start_time = time.time()

    print('Epoch : {}'.format(epoch+1))

    trainLoss = 0
    correct = 0

    lstm_model.train()

    for i in range(len(x_train_text)):

        lstm_model.zero_grad()

        tweet = torch.FloatTensor(x_train_text[i])
        label = torch.FloatTensor( np.array([y_train[i]]) )

        if cuda0 != None:
            tweet = tweet.cuda()
            label = label.cuda()

        pred = lstm_model(tweet)       
        loss = criterion_text(pred, label)

        lambdaParam = torch.tensor(0.001)
        l2_reg = torch.tensor(0.)

        if cuda0 != None:
          lambdaParam = lambdaParam.cuda()
          l2_reg = l2_reg.cuda() 

        for param in lstm_model.parameters():
          if cuda0 != None:
            l2_reg += torch.norm(param).cuda()
          else:
            l2_reg += torch.norm(param)

        loss += lambdaParam * l2_reg

        pred = pred.item()*lstm_model_weight
        
        if pred > 0.5:
            pred = 1
        else:
            pred = 0

        if pred == int( label.item() ):
            correct += 1

        trainLoss += loss.item()

        optimizer_text.zero_grad()
        loss.backward()
        optimizer_text.step()

        if (i+1)%1000 == 0:
            print('Processed {} tweets out of {}'.format(i+1, len(x_train_text)))

    train_losses.append(trainLoss/len(x_train_text))
    train_accuracies.append( correct/len(x_train_text) )

    valLoss = 0
    correct = 0

    lstm_model.eval()

    with torch.no_grad():

        for i in range(len(x_val_text)):

            tweet = torch.FloatTensor(x_val_text[i])
            label = torch.FloatTensor( np.array([y_val[i]]) )

            if cuda0 != None:
                tweet = tweet.cuda()
                label = label.cuda()

            pred = lstm_model( tweet )

            loss = criterion_text(pred, label)

            valLoss += loss.item()

            pred = pred.item()*lstm_model_weight

            if pred > 0.5:
                pred = 1
            else:
                pred = 0

            if pred == int( label.item() ):
                correct += 1

    val_losses.append(valLoss/len(x_val_text))
    val_accuracies.append( correct/len(x_val_text) )

    print('Epoch Summary:')
    print(f'Train Loss: {train_losses[-1]:7.2f}  Train Accuracy: {train_accuracies[-1]*100:6.3f}%')
    print(f'Validation Loss: {val_losses[-1]:7.2f}  Validation Accuracy: {val_accuracies[-1]*100:6.3f}%')
    print(f'Duration: {time.time() - epoch_start_time:.0f} seconds')
    print('')

    scheduler_text.step()

print(f'\nDuration: {time.time() - start_time:.0f} seconds')

Epoch : 1
Processed 1000 tweets out of 7273
Processed 2000 tweets out of 7273
Processed 3000 tweets out of 7273
Processed 4000 tweets out of 7273
Processed 5000 tweets out of 7273
Processed 6000 tweets out of 7273
Processed 7000 tweets out of 7273
Epoch Summary:
Train Loss:    0.48  Train Accuracy: 80.943%
Validation Loss:    0.44  Validation Accuracy: 79.706%
Duration: 87 seconds

Epoch : 2
Processed 1000 tweets out of 7273
Processed 2000 tweets out of 7273
Processed 3000 tweets out of 7273
Processed 4000 tweets out of 7273
Processed 5000 tweets out of 7273
Processed 6000 tweets out of 7273
Processed 7000 tweets out of 7273
Epoch Summary:
Train Loss:    0.42  Train Accuracy: 84.587%
Validation Loss:    0.43  Validation Accuracy: 81.176%
Duration: 80 seconds

Epoch : 3
Processed 1000 tweets out of 7273
Processed 2000 tweets out of 7273
Processed 3000 tweets out of 7273
Processed 4000 tweets out of 7273
Processed 5000 tweets out of 7273
Processed 6000 tweets out of 7273
Processed 7000 t